# 자동차 이미지들을 학습 및 테스트합니다.

훈련 데이터는 각 480개씩 총 1920개로 구성되어있고,
시험 데이터는 각 160개씩 총 480개로 구성되어있습니다.

차종은 세단, 쿠페, SUV, 픽업트럭으로 네가지를 구분하였고 종류별 숫자 코드는 다음과 같습니다.
0 = Sedan, 1 = Coupe, 2 = SUV, 3 = Pickup-Truck

In [30]:
import os, random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

    Namin Neural Network
    ----------
    Input Layer: 100*100*3(Input Size)
    
    1st Layer: Conv1(3EA, 3*3*3, Strides=3, Padding=1),
               ReLU1(34*34*3),
               Pool1(2*2, Strides=1),
               Bn1(33*33*3)
    2nd Layer: Conv2(6EA, 6*6*3, Strides=1, Padding=VALID),
               ReLU2(28*28*6),
               Pool2(2*2, Strides=2),
               Bn2(14*14*6)
    3rd Layer: Conv3(9EA, 3*3*6, Strides=3, Padding=2),
               ReLU3(6*6*9),
               Pool3(2*2, Strides=2),
               Bn3(3*3*9)
    4th Layer: Flatten,
               Affine(W=3*3*9, B=9)
                
    Output Layer: Softmax Cross Entropy(Sedan, Coupe, SUV, Pickup-Truck)

In [31]:
# Load to Sedan, Coupe, SUV, PickupTruck Images
trainlist, testlist = [], []
with open('train.txt') as f:
    for line in f:
        tmp = line.strip().split()
        trainlist.append([tmp[0], tmp[1]])
        
with open('test.txt') as f:
    for line in f:
        tmp = line.strip().split()
        testlist.append([tmp[0], tmp[1]])

In [32]:
# Image Preprocessing
IMG_H = 100
IMG_W = 100
IMG_C = 3

def readimg(path):
    img = plt.imread(path)
    return img

def batch(path, batch_size):
    img, label, paths = [], [], []
    for i in range(batch_size):
        img.append(readimg(path[0][0]))
        label.append(int(path[0][1]))
        path.append(path.pop(0))
        
    return img, label

In [33]:
# Neural Network
num_class = 4 # Sedan, Coupe, SUV, PickupTruck

with tf.Graph().as_default() as g:
    X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C]) # Input Layer, X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C])
    Y = tf.placeholder(tf.int32, [None]) # Y = tf.placeholder(tf.int32, [None])
    
    with tf.variable_scope('CNN'):
        # 1st Layer(Conv1 - relu1 - maxpool1 - bn1) = 33*33*3
        conv1 = tf.layers.conv2d(X, 3, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(conv1, 2, (1, 1), padding='VALID')
        bn1 = tf.compat.v1.layers.batch_normalization(pool1, training=True)
        # 2nd Layer(Conv2 - relu2 - maxpool2 - bn2) = 14*14*6
        conv2 = tf.layers.conv2d(bn1, 6, 6, (1, 1), padding='VALID', activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(conv2, 2, (2, 2), padding='VALID')
        bn2 = tf.compat.v1.layers.batch_normalization(pool2, training=True)
        # 3rd Layer(Conv3 - relu3 - maxpool3 - bn3) = 3*3*9
        conv3 = tf.layers.conv2d(bn2, 9, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(conv3, 2, (2, 2), padding='VALID')
        bn3 = tf.compat.v1.layers.batch_normalization(pool3, training=True)
        # Fully Connected Layer(Affine)
        affine1 = tf.layers.flatten(bn3)
        # Output Layer
        output = tf.layers.dense(affine1, num_class)
        
    # Softmax with Loss
    with tf.variable_scope('Loss'):
        Loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= Y, logits=output))
    
    # Training with Adam    
    train_step = tf.train.AdamOptimizer(0.005, beta1=0.9, beta2=0.999).minimize(Loss) 
    saver = tf.train.Saver()
    
    tf.summary.scalar('Epoch-Loss', Loss)
    merged = tf.summary.merge_all()

In [34]:
# Size
np.sum([np.product(var.shape) for var in g.get_collection('trainable_variables')]).value

1417

In [35]:
# Setting Batch with Training
batch_size = 48
epoch = 1000

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('tf_board', sess.graph)
    for i in range(epoch):
        batch_data, batch_label = batch(trainlist, batch_size)     
        _, loss, summary = sess.run([train_step, Loss, merged], feed_dict = {X: batch_data, Y: batch_label})
        print("Epoch:",i+1,"Loss:",loss)
        if i % 10 == 0:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)
        elif i+1 == epoch:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)

Epoch: 1 Loss: 1.842403
Epoch: 2 Loss: 1.5546517
Epoch: 3 Loss: 1.5860039
Epoch: 4 Loss: 1.4614025
Epoch: 5 Loss: 1.4471499
Epoch: 6 Loss: 1.1652559
Epoch: 7 Loss: 1.1132253
Epoch: 8 Loss: 0.9840789
Epoch: 9 Loss: 1.0310172
Epoch: 10 Loss: 0.95310575
Epoch: 11 Loss: 2.3206503
Epoch: 12 Loss: 2.5604267
Epoch: 13 Loss: 2.1791894
Epoch: 14 Loss: 2.1872263
Epoch: 15 Loss: 2.0629036
Epoch: 16 Loss: 1.8591622
Epoch: 17 Loss: 1.882165
Epoch: 18 Loss: 1.6168758
Epoch: 19 Loss: 1.5575806
Epoch: 20 Loss: 1.6853269
Epoch: 21 Loss: 2.6511672
Epoch: 22 Loss: 2.395547
Epoch: 23 Loss: 2.1813936
Epoch: 24 Loss: 1.8824915
Epoch: 25 Loss: 1.7863803
Epoch: 26 Loss: 1.7270676
Epoch: 27 Loss: 1.5662298
Epoch: 28 Loss: 1.575847
Epoch: 29 Loss: 1.671554
Epoch: 30 Loss: 1.766152
Epoch: 31 Loss: 2.5231733
Epoch: 32 Loss: 2.5118606
Epoch: 33 Loss: 2.4094818
Epoch: 34 Loss: 2.4296525
Epoch: 35 Loss: 2.0789087
Epoch: 36 Loss: 2.0049827
Epoch: 37 Loss: 1.9325482
Epoch: 38 Loss: 1.782046
Epoch: 39 Loss: 1.6473004
E

Epoch: 310 Loss: 1.4336394
Epoch: 311 Loss: 1.3938466
Epoch: 312 Loss: 1.4446888
Epoch: 313 Loss: 1.4007746
Epoch: 314 Loss: 1.5125831
Epoch: 315 Loss: 1.3854259
Epoch: 316 Loss: 1.3761324
Epoch: 317 Loss: 1.262642
Epoch: 318 Loss: 1.3508339
Epoch: 319 Loss: 1.2268624
Epoch: 320 Loss: 1.2774855
Epoch: 321 Loss: 1.4461813
Epoch: 322 Loss: 1.4722306
Epoch: 323 Loss: 1.4223171
Epoch: 324 Loss: 1.4990724
Epoch: 325 Loss: 1.4077059
Epoch: 326 Loss: 1.5642058
Epoch: 327 Loss: 1.4745747
Epoch: 328 Loss: 1.4726442
Epoch: 329 Loss: 1.4534904
Epoch: 330 Loss: 1.4098631
Epoch: 331 Loss: 1.3960013
Epoch: 332 Loss: 1.339235
Epoch: 333 Loss: 1.4161547
Epoch: 334 Loss: 1.2997531
Epoch: 335 Loss: 1.3385109
Epoch: 336 Loss: 1.3523113
Epoch: 337 Loss: 1.3570557
Epoch: 338 Loss: 1.3331851
Epoch: 339 Loss: 1.3085216
Epoch: 340 Loss: 1.300516
Epoch: 341 Loss: 1.4825948
Epoch: 342 Loss: 1.4937544
Epoch: 343 Loss: 1.5103111
Epoch: 344 Loss: 1.4784724
Epoch: 345 Loss: 1.440381
Epoch: 346 Loss: 1.4094623
Epoch

Epoch: 616 Loss: 1.3226149
Epoch: 617 Loss: 1.3576604
Epoch: 618 Loss: 1.2742294
Epoch: 619 Loss: 1.2304271
Epoch: 620 Loss: 1.2896446
Epoch: 621 Loss: 1.5077451
Epoch: 622 Loss: 1.3989416
Epoch: 623 Loss: 1.4130138
Epoch: 624 Loss: 1.3779248
Epoch: 625 Loss: 1.3087617
Epoch: 626 Loss: 1.3747567
Epoch: 627 Loss: 1.3434759
Epoch: 628 Loss: 1.3690033
Epoch: 629 Loss: 1.2837454
Epoch: 630 Loss: 1.3572639
Epoch: 631 Loss: 1.13877
Epoch: 632 Loss: 1.2873412
Epoch: 633 Loss: 1.2347803
Epoch: 634 Loss: 1.2556905
Epoch: 635 Loss: 1.2140104
Epoch: 636 Loss: 1.2804316
Epoch: 637 Loss: 1.1283599
Epoch: 638 Loss: 1.2590669
Epoch: 639 Loss: 1.1057976
Epoch: 640 Loss: 1.1547545
Epoch: 641 Loss: 1.3802239
Epoch: 642 Loss: 1.4510784
Epoch: 643 Loss: 1.3676715
Epoch: 644 Loss: 1.425223
Epoch: 645 Loss: 1.380006
Epoch: 646 Loss: 1.4575762
Epoch: 647 Loss: 1.4023453
Epoch: 648 Loss: 1.3938264
Epoch: 649 Loss: 1.402716
Epoch: 650 Loss: 1.3882693
Epoch: 651 Loss: 1.3074849
Epoch: 652 Loss: 1.235497
Epoch: 

Epoch: 922 Loss: 1.4446793
Epoch: 923 Loss: 1.367886
Epoch: 924 Loss: 1.3973771
Epoch: 925 Loss: 1.376578
Epoch: 926 Loss: 1.3759985
Epoch: 927 Loss: 1.3392767
Epoch: 928 Loss: 1.3446951
Epoch: 929 Loss: 1.3332747
Epoch: 930 Loss: 1.3331488
Epoch: 931 Loss: 1.2406961
Epoch: 932 Loss: 1.1870726
Epoch: 933 Loss: 1.2694283
Epoch: 934 Loss: 1.1398345
Epoch: 935 Loss: 1.1658357
Epoch: 936 Loss: 1.2553042
Epoch: 937 Loss: 1.2879639
Epoch: 938 Loss: 1.148333
Epoch: 939 Loss: 1.1609797
Epoch: 940 Loss: 1.1967245
Epoch: 941 Loss: 1.4417938
Epoch: 942 Loss: 1.3122683
Epoch: 943 Loss: 1.3197153
Epoch: 944 Loss: 1.2993752
Epoch: 945 Loss: 1.3003615
Epoch: 946 Loss: 1.3240132
Epoch: 947 Loss: 1.2966242
Epoch: 948 Loss: 1.2969718
Epoch: 949 Loss: 1.2173263
Epoch: 950 Loss: 1.2738047
Epoch: 951 Loss: 1.0547949
Epoch: 952 Loss: 1.3322738
Epoch: 953 Loss: 1.2231416
Epoch: 954 Loss: 1.1523916
Epoch: 955 Loss: 1.1437947
Epoch: 956 Loss: 1.2130675
Epoch: 957 Loss: 1.083025
Epoch: 958 Loss: 1.1299263
Epoch

In [36]:
# Print an Accuracy
acc = 0

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(len(testlist)):
        batch_data, batch_label = batch(testlist, 1)
        logit = sess.run(output, feed_dict = {X:batch_data})
        if np.argmax(logit[0]) == batch_label[0]:
            acc += 1
        else:
            print(logit[0], batch_label[0])
            
    print("Accuracy:", acc/len(testlist))

INFO:tensorflow:Restoring parameters from logs\model.ckpt-1000
[-0.28698385 -0.321227    0.6881922  -1.2158612 ] 0
[0.38163096 0.57300365 1.2239884  1.7631562 ] 0
[-0.38732666  0.11303855 -0.66456974 -0.4872252 ] 0
[ 0.27771845 -0.92172414  0.8993701   1.9032156 ] 0
[0.41678143 0.6598618  0.32812378 1.4228878 ] 0
[-0.8118152  3.362328  -1.3685751 -2.3973255] 0
[-0.8321893  -1.9414852   0.10708249  0.16838491] 0
[-0.34275898  0.9142803  -0.5197569  -3.1513398 ] 0
[-1.0745963  -2.239171    0.06340072  0.24595931] 0
[-0.58498126  1.5430027   0.45462608 -0.2618784 ] 0
[-1.0393459  -1.7565814  -0.00444157  0.5721166 ] 0
[-0.97900563 -0.98945546  0.4478366  -2.2484117 ] 0
[-8.1724495e-01 -1.9470155e-03 -2.6080048e-01 -2.7088356e+00] 0
[-0.45428878  1.4320624   0.43006253 -2.1133294 ] 0
[-1.0395371  -0.7818316  -0.41779605 -1.6435171 ] 0
[ 0.11837521 -1.2139616   1.4864583   0.9747925 ] 0
[-0.04425101 -1.1167481   1.6856391   1.2936469 ] 0
[-0.24951127 -1.5698161   1.9690194   2.0473669 ] 0
[

[-0.1366646  -3.0690105   0.35708657  0.00645146] 1
[ 0.7127959   0.08411755 -1.128293   -0.16971469] 1
[-0.49606207  0.14563558  0.94002485 -1.9787498 ] 1
[1.5293674  0.13882717 0.36578894 1.144209  ] 1
[0.53316456 0.2755158  0.59671736 1.2872816 ] 1
[-0.04453268 -0.83661205 -0.5412307  -0.1924164 ] 1
[-1.7382385  -1.2629309   0.6838681   0.37062237] 1
[ 0.7949715  -1.3670774   0.7875375   0.80123496] 1
[0.95726943 0.50326556 0.568724   1.1781058 ] 1
[-0.7708544   0.1442288   0.38106808 -0.9897845 ] 1
[ 1.1261288   0.4134846  -0.41955036  2.0258892 ] 1
[-0.1707908  -1.9075913   1.3939202  -0.49799618] 1
[-0.6596896  -0.01977901  0.32743683  1.5660088 ] 1
[-0.82517    -0.75463206 -0.6573956  -1.4559885 ] 1
[-1.2461475  -0.8743057  -0.05832357 -1.302405  ] 1
[-1.1645391  -1.0519586   0.68321353 -1.4958282 ] 1
[-0.8020095  -1.0423573   0.00338017 -1.5179584 ] 1
[-0.3620492 -1.2691157  1.053124   1.3596843] 1
[-1.9050999  -0.9422445  -0.4442162  -0.81590223] 1
[ 1.5269262  -1.1096115   0.

[0.6338743  0.56043863 1.3497678  0.10598293] 3
[ 0.02806809  2.401536   -1.333574   -1.3800466 ] 3
[-0.11178063 -1.174769   -0.8321129  -1.5025595 ] 3
[0.31847858 0.00793937 1.6276566  0.7149265 ] 3
[ 1.711208   -1.6452326   0.73644906  1.467462  ] 3
[-1.3687611   1.9563527  -0.24890503  0.00313404] 3
[-0.40609536  0.13945712 -0.57430375 -0.55364966] 3
[-1.4539045   0.59922427 -0.66204697 -1.831228  ] 3
[ 1.1975547 -0.2907412  0.3282321  1.0352954] 3
[ 0.36550868  0.08581685 -0.6377894  -0.39466226] 3
[ 0.04727875  0.21168382  0.7442707  -0.9102658 ] 3
[ 0.25376248  1.5480411  -0.40401822 -0.5419005 ] 3
[ 1.3658872   0.74498373 -0.9376021   0.33338615] 3
[-0.15298444 -0.35294682  1.580652    0.79616094] 3
[-0.15135212 -1.060815    1.4471433   0.26550758] 3
[-0.7118351   0.6424823  -1.1035107   0.26985162] 3
[-1.6807731  -0.3798003   0.57140183 -1.4451029 ] 3
[ 0.6869642   0.46116883  0.97941256 -0.04115868] 3
[-0.3369592   0.8344749   0.7524578   0.01157093] 3
[-0.09722108 -1.2106944 